In [1]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importing Libraries
import os
import pandas as pd
import numpy as np
import psycopg2

In [7]:

# Define database connection parameters
db_params = {
    "database": "Super_Store",
    "user": "postgres",
    "password": "1234",
    "host": "localhost",
    "port": "5432"
}

In [53]:
# Excel file path
excel_file_path = "D:\SQL_Queries\Superstore.xlsx"

# Sheet name
sheet_name = "People"

# Table name
table_name = "People"

# Read Excel data using pandas with the openpyxl engine
excel_data = pd.read_excel(excel_file_path, sheet_name=sheet_name, engine='openpyxl')

In [54]:
 excel_data.columns = [x.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace(".","_").replace("$","").replace("%","") for x in excel_data.columns]

In [55]:
# Extract column names and data types
column_names = excel_data.columns.tolist()

In [56]:
excel_data.dtypes

person    object
region    object
dtype: object

In [57]:
 excel_data.columns

Index(['person', 'region'], dtype='object')

In [58]:
# Convert pandas data types to PostgreSQL data types
data_type_mapping = {
     'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'int',  #bigint for big values, can be changed with int for small values
        'datetime64[ns]': 'timestamp' 
}
data_types = [data_type_mapping[str(data_type)] for data_type in excel_data.dtypes]
# Establish a connection to the database
conn = psycopg2.connect(**db_params)

# Create a cursor
cursor = conn.cursor()


In [59]:
 cursor.execute("drop table if exists %s;" % (table_name))

In [60]:
# Construct CREATE TABLE query dynamically
create_table_query = f"CREATE TABLE {table_name} ("
for col_name, data_type in zip(column_names, data_types):
    create_table_query += f"{col_name} {data_type}, "
create_table_query = create_table_query.rstrip(", ") + ")"
cursor.execute(create_table_query)
conn.commit()


In [61]:
# Insert data into the table
for _, row in excel_data.iterrows():
    insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s']*len(column_names))})"
    values = tuple(row)
    cursor.execute(insert_query, values)
    conn.commit()

# Close the cursor and the connection
cursor.close()
conn.close()
